<a href="https://colab.research.google.com/github/Benjamin1118/DS-Unit-2-Linear-Models/blob/master/module_3_Benjamin_Bishop_assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
#importing all the libraries I will need for this... I think...
import numpy as np
import category_encoders as ce 
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from datetime import datetime 
from IPython.display import display, HTML

In [0]:
#df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'].astype(str),format= '%m/%d/%Y')

In [0]:
#How to change to MONTH DD YYYY
#df['SALE_DATE'] = df['SALE_DATE'].dt.strftime('%b %d %Y')

In [0]:
#setting up test and training data. 
train = df[df['SALE_DATE']<= '2019-03-31']
test = df[df['SALE_DATE'] > '2019-03-31']

In [0]:
X_train= train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]

In [99]:
target = 'SALE_PRICE'
high_card = ['NEIGHBORHOOD', 'LAND_SQUARE_FEET', 'BUILDING_CLASS_CATEGORY', 'EASE-MENT', \
             'BUILDING_CLASS_AT_PRESENT', 'ADDRESS','APARTMENT_NUMBER',\
             'BUILDING_CLASS_AT_TIME_OF_SALE','SALE_DATE']
features= train.columns.drop([target]+ high_card)
features

Index(['BOROUGH', 'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE'],
      dtype='object')

In [100]:
df.head()


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,FASHION,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,FASHION,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,GREENWICH VILLAGE-WEST,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [101]:
X_train

,BOROUGH,TAX_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
0,1,2,716,1246,10011.0,1.0,0.0,1.0,1979.0,2007.0,2
1,1,4,812,68,10018.0,0.0,6.0,6.0,15435.0,1920.0,4
2,1,4,839,69,10018.0,0.0,7.0,7.0,11332.0,1930.0,4
3,1,2,592,1041,10014.0,1.0,0.0,1.0,500.0,0.0,2
4,1,2C,1379,1402,10065.0,1.0,0.0,1.0,6406.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...
23035,4,1,10965,276,11429.0,1.0,0.0,1.0,1224.0,1945.0,1
23036,4,2,169,29,11104.0,0.0,0.0,0.0,0.0,1929.0,2
23037,4,2,131,4,11377.0,0.0,0.0,0.0,0.0,1932.0,2
23038,4,1,8932,18,11421.0,2.0,1.0,3.0,2200.0,1931.0,1


In [0]:
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train= encoder.fit_transform(X_train)

In [0]:
X_test= encoder.transform(X_test)

In [104]:
X_train

,BOROUGH_1,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,TAX_CLASS_AT_PRESENT_2,TAX_CLASS_AT_PRESENT_4,TAX_CLASS_AT_PRESENT_2C,TAX_CLASS_AT_PRESENT_2B,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1B,TAX_CLASS_AT_PRESENT_2A,TAX_CLASS_AT_PRESENT_1A,TAX_CLASS_AT_PRESENT_1C,TAX_CLASS_AT_PRESENT_1D,TAX_CLASS_AT_PRESENT_nan,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,716,1246,10011.0,1.0,0.0,1.0,1979.0,2007.0,2
1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,812,68,10018.0,0.0,6.0,6.0,15435.0,1920.0,4
2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,839,69,10018.0,0.0,7.0,7.0,11332.0,1930.0,4
3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,592,1041,10014.0,1.0,0.0,1.0,500.0,0.0,2
4,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1379,1402,10065.0,1.0,0.0,1.0,6406.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23035,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,10965,276,11429.0,1.0,0.0,1.0,1224.0,1945.0,1
23036,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,169,29,11104.0,0.0,0.0,0.0,0.0,1929.0,2
23037,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,131,4,11377.0,0.0,0.0,0.0,0.0,1932.0,2
23038,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,8932,18,11421.0,2.0,1.0,3.0,2200.0,1931.0,1


In [0]:
#select best cotegory
selector = SelectKBest(score_func=f_regression, k=15)

X_train_selected= selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
X_test_selected.shape, X_test_selected.shape


In [107]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [0]:
for alpha in [.001, .01, .1, 1, 100, 1000 ]:
    # Fit Ridge Regression model
    display(HTML(f'Ridge Regression, with alpha={alpha}'))
    model = Ridge(alpha=alpha, normalize=True)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Get Test MAE
    mae = mean_absolute_error(y_test, y_pred)
    display(HTML(f'Test Mean Absolute Error: ${mae:,.0f}'))
    
    # Plot coefficients
    coefficients = pd.Series(model.coef_, X_train.columns)
    plt.figure(figsize=(16,8))
    coefficients.sort_values().plot.barh(color='grey')
    plt.xlim(-1000,1500)
    plt.show()

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
#top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
#df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'